In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import librosa
import csv

yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

from google.colab import files
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

waveform, _ = librosa.load(audio_file, sr=16000, mono=True)
waveform = waveform.astype(np.float32)

scores, _, spectrogram = yamnet_model(waveform)

class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names = []
with tf.io.gfile.GFile(class_map_path) as class_map_file:
    reader = csv.reader(class_map_file)
    next(reader)
    for row in reader:
        class_names.append(row[2])

top_n = 5
top_class_indices = np.argsort(scores[0])[::-1][:top_n]
for i in top_class_indices:
    print(f'{class_names[i]}: {scores[0][i]}')

    if class_names[i].lower() == 'gunshot' and scores[0][i] > 0.5:
        print(f"Gunshot detected with probability: {scores[0][i]}")

plt.figure(figsize=(10, 4))
plt.imshow(spectrogram.numpy().T, aspect='auto', interpolation='nearest', origin='lower')
plt.title('Spectrogram')
plt.xlabel('Frames')
plt.ylabel('Frequency Bands')
plt.show()
